In [ ]:
# Downloading images

import csv
import urllib.request
import os

# Define the CSV file name
csv_file_name = ('/home/apiiit123/ml/student_resource_3/dataset/train.csv')

# Create a directory to save downloaded images
download_directory = 'downloaded_images'
os.makedirs(download_directory, exist_ok=True)

# Open the CSV file and read the image URLs
with open('/home/apiiit123/ml/student_resource_3/dataset/train.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # Assuming the URL is in a column named 'image_link'
        url = row['image_link']
        
        # Create a filename based on another column (e.g., 'filename')
        filename = row.get('filename', 'default_image')  # Use 'default_image' if 'filename' is not provided
        file_path = os.path.join(download_directory, f"{filename}.jpg")
        
        try:
            # Download the image
            urllib.request.urlretrieve(url, file_path)
            print(f"Downloaded: {file_path}")
        except Exception as e:
            print(f"Failed to download {url}. Reason: {e}")

print("Image download process completed.")

Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg
Downloaded: downloaded_images/default_image.jpg


In [ ]:
# Load an image
from PIL import Image

# Load the image
img = Image.open('/home/apiiit123/ml/downloaded_images/default_image.jpg')

# Display the image
img.show()

In [ ]:
# Image Preprocessing
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Define the directory containing images
image_dir = '/home/apiiit123/ml/downloaded_images'  # Change this to your image directory
processed_images_dir = 'processed_images'
os.makedirs(processed_images_dir, exist_ok=True)

# Parameters
target_size = (150, 150)  # Desired size for resizing
batch_size = 32  # Batch size for data augmentation

# Function to preprocess and save images
def preprocess_images(image_dir):
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(image_dir, filename)
            # Load image
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize to [0, 1]
            
            # Save processed image
            processed_img_path = os.path.join(processed_images_dir, filename)
            cv2.imwrite(processed_img_path, img_array * 255)  # Convert back to [0, 255] for saving

# Preprocess images
preprocess_images(image_dir)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Example of using the data generator
sample_image = load_img(os.path.join(processed_images_dir, 'default_image.jpg'), target_size=target_size)
sample_image_array = img_to_array(sample_image)
sample_image_array = np.expand_dims(sample_image_array, axis=0)

# Generate augmented images
augmented_images = datagen.flow(sample_image_array, batch_size=1)

# Save augmented images
for i in range(5):  # Generate and save 5 augmented images
    augmented_image = next(augmented_images)[0]
    cv2.imwrite(os.path.join(processed_images_dir, f'augmented_image_{i}.jpg'), augmented_image)

print("Image preprocessing and augmentation complete.")

In [2]:
import pytesseract
from PIL import Image, ImageFilter
import cv2
import numpy as np

# Set the path to the Tesseract executable
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# Load the image using OpenCV
image_path = '/home/apiiit123/ml/downloaded_images/default_image.jpg'  # Replace with your image path
image = cv2.imread(image_path)
    
# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
# Apply Gaussian Blur to reduce noise
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    
# Use adaptive thresholding to binarize the image
binary_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                         cv2.THRESH_BINARY, 11, 2)
    
# Save the processed image (optional, for debugging)
cv2.imwrite('processed_image.jpg', binary_image)
    
# Use Tesseract to extract text from the processed image
text = pytesseract.image_to_string(binary_image)
    
# Print the extracted text
print("Extracted Text:")
print(text)

Extracted Text:
EASY wild
SQUEEZY!

168 DROPPERS 1DROPPERFUL f{ 3) 56-DAY

per bottle (1 bulb squeeze)
serving size

 

     
  

SUPPLEMENT FACTS
Serving size: 0.7 ml / Servings about 168
Amount Per Serving %DV

Organic Mullein Leaf 679mg *
(Verbascum thapsus)

* Daily Value (DV) not established

Other Ingredients: Certified Organic
vegetable glycerin, distilled water.
Does not contain: Alcohol, Sugar.

 
     
 



In [11]:
import pandas as pd
# Load datasets
train_data = pd.read_csv('/home/apiiit123/ml/train1.csv')
test_data = pd.read_csv('/home/apiiit123/ml/test1.csv')

In [21]:
# Define constants
ALLOWED_UNITS = {'gram', 'centimetre', 'millilitre', 'kilogram', 'millimetre', 'ounce', 'litre'}

In [23]:
# Function to clean and process extracted text
    # Regex to find numbers followed by units
pattern = r'(\d+\.?\d*)\s*(gram|g|cm|centimeter|ml|ounce|kg|kilogram|litre|mm)'
matches = re.findall(pattern, text.lower())  # Find all matches
    
cleaned_results = []
    
for match in matches:
    number = match[0]
    unit = match[1]
    # Map shorthand to allowed unit
    unit_mapping = {
        'g': 'gram',
        'cm': 'centimetre',
        'ml': 'millilitre',
        'kg': 'kilogram',
        'mm': 'millimetre',
        'ounce': 'ounce',
        'litre': 'litre'
    }
    unit = unit_mapping.get(unit, unit)
    if unit in ALLOWED_UNITS:
        cleaned_results.append(f"{number} {unit}")
    
# Ensure that the result aligns with the entity name
if entity_name == 'item_weight':
    # Example: refine result if entity_name suggests weight
    if cleaned_results:
        print(cleaned_results[0])
elif entity_name == 'item_volume':
    # Example: refine result if entity_name suggests volume
    if cleaned_results:
        print(cleaned_results[0])
# Add more conditions based on entity_name as needed

print(cleaned_results[0] if cleaned_results else "")

NameError: name 'entity_name' is not defined

In [13]:
# Main processing loop for the test dataset
predictions = []
for idx, row in test_data.iterrows():
    image_url = row['image_link']
    img = download_image(image_url)
    
    if img:
        extracted_text = extract_text_from_image(image_url)
        cleaned_text = clean_extracted_text(extracted_text, row['entity_name'])
        
        prediction = cleaned_text
        
        # Append the prediction to the list
        predictions.append({"index": row['index'], "prediction": prediction})
    else:
        predictions.append({"index": row['index'], "prediction": ""})
# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions)
# Save the predictions to CSV in the required format
output_file = '/home/rguktrkvalley/Desktop/test_out.csv'
predictions_df.to_csv(output_file, index=False)
# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions)

print(f'Predictions saved to {output_file}')

NameError: name 'download_image' is not defined